### Import das dependências

In [1]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### 1. Importar o dataset - "Número de ordens de serviço remotas realizadas"

In [2]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

diagrama_de_carga_path = data_path + '22-diagrama-de-carga-por-instalacao.csv'

df_diagrama_de_carga = pd.read_csv(diagrama_de_carga_path, delimiter=';')

### 2. Ver o dataset

In [3]:
df_diagrama_de_carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67960 entries, 0 to 67959
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Ano                           67960 non-null  int64 
 1   Mês                           67960 non-null  int64 
 2   data                          67960 non-null  object
 3   Distrito                      67960 non-null  object
 4   Concelho                      67960 non-null  object
 5   Freguesia                     67960 non-null  object
 6   Inclui Mobilidade Elétrica    67960 non-null  object
 7   CPEs com DCs Recolhidos (#)   67960 non-null  int64 
 8   CodDistritoConcelhoFreguesia  67960 non-null  object
dtypes: int64(3), object(6)
memory usage: 4.7+ MB


1. Verificar se há elementos null

In [4]:
# Verificar nulls
null_counts = df_diagrama_de_carga.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

df_diagrama_de_carga.head(15)

Número de nulls por coluna:
Ano                             0
Mês                             0
data                            0
Distrito                        0
Concelho                        0
Freguesia                       0
Inclui Mobilidade Elétrica      0
CPEs com DCs Recolhidos (#)     0
CodDistritoConcelhoFreguesia    0
dtype: int64


,Ano,Mês,data,Distrito,Concelho,Freguesia,Inclui Mobilidade Elétrica,CPEs com DCs Recolhidos (#),CodDistritoConcelhoFreguesia
0,2023,8,2023-08,Bragança,Mirandela,Fradizela,Não,83,040714
1,2023,8,2023-08,Bragança,Mirandela,Vale de Telhas,Não,147,040734
2,2023,8,2023-08,Bragança,Mogadouro,Tó,Não,87,040819
3,2023,8,2023-08,Bragança,Vila Flor,Freixiel,Não,29,041005
4,2023,8,2023-08,Castelo Branco,Castelo Branco,Salgueiro do Campo,Não,21,050220
5,2023,8,2023-08,Castelo Branco,Oleiros,Isna,Não,110,050605
6,2023,8,2023-08,Coimbra,Figueira da Foz,Lavos,Sim,1,060522
7,2023,8,2023-08,Coimbra,Figueira da Foz,Paião,Não,235,060523
8,2023,8,2023-08,Coimbra,Lousã,Gândaras,Não,580,060706
9,2023,8,2023-08,Coimbra,Oliveira do Hospital,União das freguesias de Ervedal e Vila Franca ...,Não,111,061122


### 3. Tratamento e agregação dos dados

1. Remover atributos como a data, CodDistritoConcelhoFreguesia
2. Agrupar por concelhos e remover a freguesia
3. Agrupar por semestres e anos

1. Remover atributos como a data, CodDistritoConcelhoFreguesia

In [5]:
# Remover atributos como a data e código, distrito
df_diagrama_de_carga.drop(columns=['data', 'CodDistritoConcelhoFreguesia', 'Distrito'], inplace=True)
df_diagrama_de_carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67960 entries, 0 to 67959
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Ano                          67960 non-null  int64 
 1   Mês                          67960 non-null  int64 
 2   Concelho                     67960 non-null  object
 3   Freguesia                    67960 non-null  object
 4   Inclui Mobilidade Elétrica   67960 non-null  object
 5   CPEs com DCs Recolhidos (#)  67960 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 3.1+ MB


2. Agrupar por concelhos e remover a freguesia

In [6]:
df_agrupado = df_diagrama_de_carga.groupby(['Concelho', 'Ano', 'Mês', 'Inclui Mobilidade Elétrica']).sum().reset_index()
df_agrupado.drop(columns=['Freguesia'], inplace=True)
df_agrupado.rename(columns={'Concelho':'concelho', 'Ano':'ano', 'Mês':'mes','Inclui Mobilidade Elétrica': 'inclui_mobilidade_eletrica', 'CPEs com DCs Recolhidos (#)':'cpes_com_dcs_recolhidos_quantidade'}, inplace=True)
df_agrupado.head()

,concelho,ano,mes,inclui_mobilidade_eletrica,cpes_com_dcs_recolhidos_quantidade
0,Abrantes,2022,8,Não,11097
1,Abrantes,2022,8,Sim,1
2,Abrantes,2022,9,Não,11298
3,Abrantes,2022,9,Sim,1
4,Abrantes,2022,10,Não,11563


3. Verificar o número de linhas

In [7]:
# Contagem de linhas antes do agrupamento
num_linhas_antes = len(df_diagrama_de_carga)

# Contagem de linhas depois do agrupamento
num_linhas_depois = len(df_agrupado)

# Exibição das contagens
print("Número de linhas antes do agrupamento:", num_linhas_antes)
print("Número de linhas depois do agrupamento:", num_linhas_depois)

df_agrupado.info()

df_agrupado.head(15)

Número de linhas antes do agrupamento: 67960
Número de linhas depois do agrupamento: 11040
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11040 entries, 0 to 11039
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   concelho                            11040 non-null  object
 1   ano                                 11040 non-null  int64 
 2   mes                                 11040 non-null  int64 
 3   inclui_mobilidade_eletrica          11040 non-null  object
 4   cpes_com_dcs_recolhidos_quantidade  11040 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 431.4+ KB


,concelho,ano,mes,inclui_mobilidade_eletrica,cpes_com_dcs_recolhidos_quantidade
0,Abrantes,2022,8,Não,11097
1,Abrantes,2022,8,Sim,1
2,Abrantes,2022,9,Não,11298
3,Abrantes,2022,9,Sim,1
4,Abrantes,2022,10,Não,11563
5,Abrantes,2022,10,Sim,1
6,Abrantes,2022,11,Não,11969
7,Abrantes,2022,11,Sim,1
8,Abrantes,2022,12,Não,12210
9,Abrantes,2022,12,Sim,1


4. Verificar se está tudo bem

In [8]:
# Para a coleção de Consumos Mensais por Concelho
df_consumos_lisboa = df_agrupado[df_agrupado['concelho'] == 'Lisboa']
df_consumos_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, 4426 to 4465
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   concelho                            40 non-null     object
 1   ano                                 40 non-null     int64 
 2   mes                                 40 non-null     int64 
 3   inclui_mobilidade_eletrica          40 non-null     object
 4   cpes_com_dcs_recolhidos_quantidade  40 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.9+ KB


In [9]:
df_consumos_lisboa

,concelho,ano,mes,inclui_mobilidade_eletrica,cpes_com_dcs_recolhidos_quantidade
4426,Lisboa,2022,8,Não,243736
4427,Lisboa,2022,8,Sim,107
4428,Lisboa,2022,9,Não,236050
4429,Lisboa,2022,9,Sim,110
4430,Lisboa,2022,10,Não,241055
4431,Lisboa,2022,10,Sim,118
4432,Lisboa,2022,11,Não,237329
4433,Lisboa,2022,11,Sim,118
4434,Lisboa,2022,12,Não,237600
4435,Lisboa,2022,12,Sim,126


5. Guardar num ficheiro

In [10]:
file_path = data_path + 'treated/diagrama_de_carga_por_mes.csv'

df_agrupado.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)

DataFrame guardado em  ../data/treated/diagrama_de_carga_por_mes.csv
